# Selenium Basic command

1. driver.get('url') : 페이지 접속
2. elem = driver.find_element_by_name('q') : 커서를 검색어 입력 부분에 위치시킴
    - find_by_class_name or find_by_xpath 등도 가능 (xpath는 html 검사 시 우클릭>xpath copy로 복사 가능
3. elem.clear() : 검색어 부분 입력내용 삭제
4. elem.send_keys('내용') : 내용 입력
5. elem.submit() : 입력한 내용 검색 실행
    - click() : 클릭
6. driver.close() : 페이지 닫음
7. aseert 'keyword' in driver.title or driver.page_source : 제목이나 내용에 keyword가 있는지 파악 - 없으면 assertion error 반환

In [1]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from urllib.parse import quote_plus

import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [22]:
ID = 'hanshow113'
PW = 'wlsldjtm12!'

In [2]:
def instagram_login(id, pw):
    driver.get(url)
    driver.implicitly_wait(5)
    driver.find_element_by_name('username').send_keys(id)    # id 입력
    elem_pw = driver.find_element_by_name('password')    # pw 입력
    elem_pw.send_keys(pw)
    elem_pw.submit()
    
    driver.implicitly_wait(5)    # 파싱될 때까지 5초 기다림 (미리 완료되면 waiting 종료됨)
    driver.find_element_by_class_name('cmbtv').click()    # 비밀번호 저장하지 않음
    
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()    # 알림설정 무시

In [3]:
def main_search(keyword):
    search = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
    search.clear()
    search.send_keys(keyword)
    search_list1 = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]')
    search_list1.click()

In [23]:
#접속주소 설정
url = 'http://www.instagram.com'
path = 'C:\\Users\\rectworks\\Downloads\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(path)

In [24]:
instagram_login(ID, PW)

In [25]:
main_search('여기오늘')

첫 번째 게시글 클릭
driver.find_element_by_xpath('/html/body/div[1]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div').click()

In [30]:
SCROLL_PAUSE_TIME = 2.0
post_link = []

while True:
    pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
    bsObj = bs(pageString, 'lxml')

    for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
        a_len = len(postline.select('a'))
        # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
        for post in range(a_len):   
            item = postline.select('a')[post]
            link = item.attrs['href']
            post_link.append(link)



    last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
    time.sleep(SCROLL_PAUSE_TIME)
    # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
    #driver.implicitly_wait(SCROLL_PAUSE_TIME)
    # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

In [190]:
# 날짜 관계없이 게시물 내용 끝까지 수집

num_of_post = len(post_link)

print('총 {0}개의 게시글을 수집합니다.'.format(num_of_post))

id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []

for i in tqdm_notebook(range(num_of_post)):
    
    tag_list.append([])
    
    # 개별 링크 리스트 
    link_list.append("https://www.instagram.com" + post_link[i])   
    req = Request("https://www.instagram.com" + post_link[i], headers={'User-Agent': 'Mozila/5.0'})
    postpage = urlopen(req).read()
    
    post_body = bs(postpage, 'lxml', from_encoding='utf-8')
    post_core = post_body.find('meta', attrs={'property':"og:description"})
    contents = post_core['content']

    # 시간    
    posttxt = str(postpage)    
    timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
    date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))
    month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
    day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))
    
    # 좋아요
    try:
        likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
    except:
        likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
    like_list.append(likes)
    
    # 개별 계정
    if "@" and ")" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find(")")]
    elif "@" and ")" not in contents and "on Instagram" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
    else :
        personal_id = contents[1: contents.find(' posted on')]
    id_list.append(personal_id)
    
    '''    
    (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
    때문에 여러 형식별 id 추출 if문 수행
    '''
    
    # 해시태그
    for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
        hashtags = tag_content['content'].rstrip(',')
        tag_list[i].append(hashtags)


총 32개의 게시글을 수집합니다.


In [192]:
insta_dict = {'날짜':date_list,
              '월':month_list,
              '일':day_list,
              '계정':id_list,
             '좋아요':like_list,
             '해시태그':tag_list,

             '링크':link_list}
df = pd.DataFrame(insta_dict)
df

,날짜,월,일,계정,좋아요,해시태그,링크
0,2020/03/06 15:09,03,06,cc_coopcenter,13,"[홈페이지, 네이쳐앤드피플, www, 여기오늘, 춘천시협동조합지원센터]",https://www.instagram.com/p/B9YbBjupqfI/
1,2020/03/07 20:34,03,07,s.hyeyoung,52,"[흔들리지않고피는꽃이어디있으랴, 여기오늘]",https://www.instagram.com/p/B9blFm8nAnk/
2,2020/06/10 18:06,06,10,s.hyeyoung,42,"[여기행복, 노을찾아, 여기오늘, 섬진강줄기따라, 카메라, 일상, 전남구례, 대한민...",https://www.instagram.com/p/CBP7ou_nRdU/
3,2020/05/28 11:52,05,28,s.hyeyoung,44,[],https://www.instagram.com/p/CAtyeJBn90_/
4,2017/11/26 20:10,11,26,unique.h.s,6,"[여기오늘, 밤산책]",https://www.instagram.com/p/Bb9M-Z6F_Do/
5,2020/06/14 16:45,06,14,s.hyeyoung,52,"[산청, 힐링여행]",https://www.instagram.com/p/CBaFfBwnPO4/
6,2018/06/23 22:43,06,23,hong.pa,71,"[여기오늘, 힘들구만, 월드컵보러, 동중, dsm, 전세냄, 맛집, 중학교모임, 가...",https://www.instagram.com/p/BkXokh9lMF9/
7,2017/09/26 09:31,09,26,ayeoni_88,34,"[방송탔다, 양재시민의숲역, 파래김, 시노시작🖖, 서울경기, 맛스타그램, 4번출구,...",https://www.instagram.com/p/BZe_RggAkrM/
8,2020/03/18 01:00,03,18,untersbergbahn,26,[],https://www.instagram.com/p/B91zZ4BnIk0/
9,2019/07/26 00:04,07,26,eun_kyung_sin,23,"[태양이네, 동네예쁜집, 태양이장난감, 월정리해변, 애기데리고, 편안해질, 여기오늘...",https://www.instagram.com/p/B0WBZ2wnkoG/
